# qdrant 评估示例 - 使用 LlamaIndex

初步结论：

- 效果没有faiss好
- 后续看是否还有改进的余地

## 准备工作

In [1]:
%%time
%%capture

!pip install llama-index-vector-stores-qdrant
!pip install qdrant_client
!pip install llama-index-llms-openai-like
!pip install llama-index-readers-file

CPU times: user 31.6 ms, sys: 14.2 ms, total: 45.8 ms
Wall time: 8.34 s


In [2]:
%%time

# import
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client.models import Distance, VectorParams

from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings

CPU times: user 3.72 s, sys: 357 ms, total: 4.08 s
Wall time: 3.78 s


In [3]:
%%time

llm = OpenAILike(model="xiaoyu", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

Settings.llm =llm

CPU times: user 115 ms, sys: 7.45 ms, total: 122 ms
Wall time: 122 ms


In [4]:
%%time

# 初始化全局 embedding 模型
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 442 ms, sys: 7.91 ms, total: 450 ms
Wall time: 449 ms


In [5]:
%%time

Settings.chunk_size=128
Settings.chunk_overlap=10

Settings

CPU times: user 270 ms, sys: 15.8 ms, total: 286 ms
Wall time: 285 ms


_Settings(_llm=OpenAILike(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f3462e42fb0>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7f3437c81750>, completion_to_prompt=<function default_completion_to_prompt at 0x7f3437cd3f40>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='xiaoyu', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750', api_base='http://192.168.0.72:3000/v1', api_version='', context_window=3900, is_chat_model=True, is_function_calling_model=False, tokenizer=None), _embed_model=OllamaEmbedding(model_name='dztech/bge-large-zh:v1.5', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f3462e42fb0>, num_workers=None, base_url='http://19

In [6]:
%%time

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

CPU times: user 244 µs, sys: 0 ns, total: 244 µs
Wall time: 247 µs


In [7]:
%%time

# load documents
documents = SimpleDirectoryReader("./books/").load_data()

CPU times: user 10.3 ms, sys: 188 µs, total: 10.5 ms
Wall time: 9.99 ms


In [10]:
%%time

client = qdrant_client.QdrantClient(
    location=":memory:",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
    # client_kwargs={
    #     vectors_config:VectorParams(size=1024, distance=Distance.COSINE),
    # }
)

CPU times: user 315 µs, sys: 0 ns, total: 315 µs
Wall time: 356 µs


In [11]:
%%time
vector_store = QdrantVectorStore(client=client, collection_name="simple")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

CPU times: user 15.8 s, sys: 635 ms, total: 16.5 s
Wall time: 1min 21s


## 基本嵌入查询

In [12]:
%%time

# Query Data
query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.2
)

CPU times: user 33.5 ms, sys: 70 µs, total: 33.6 ms
Wall time: 33.2 ms


In [13]:
%%time

streaming_response = query_engine.query("方鸿渐的妻子是谁")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方鸿渐的妻子是孙柔嘉。
CPU times: user 220 ms, sys: 110 ms, total: 330 ms
Wall time: 14.9 s


In [14]:
%%time

streaming_response = query_engine.query("文中提到的局部真理是啥意思")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.16

In [15]:
%%time

streaming_response = query_engine.query("方鸿渐的父亲是谁，说出他的名字")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
</|system|>
CPU times: user 189 ms, sys: 130 ms, total: 320 ms
Wall time: 11.2 s


## rerank

In [16]:
%%time

from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(model='/models/bge-reranker-v2-m3', top_n=5)

CPU times: user 1.07 s, sys: 665 ms, total: 1.74 s
Wall time: 978 ms


In [17]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[reranker],
    similarity_cutoff=0.5
)

CPU times: user 174 µs, sys: 0 ns, total: 174 µs
Wall time: 178 µs


In [18]:
%%time

streaming_response = query_engine.query("方鸿渐的妻子是谁")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方鸿渐的妻子是孙柔嘉。
CPU times: user 2.04 s, sys: 361 ms, total: 2.4 s
Wall time: 3.42 s


In [19]:
%%time

streaming_response = query_engine.query("方鸿渐的父亲是谁，说出他的名字")
streaming_response.print_response_stream()
display(len(streaming_response.source_nodes))

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方鸿渐的父亲没有直接的名字提及在给定的文本中。但从上下文推测，他可能是一个有影响力的人物，因为提到韩学愈向他买文（这可能暗示他是出版商或者有文学方面的地位），并且方鸿渐提到了要对“他父亲”行礼，表明他有一定的权威。然而，具体的名字并未在给定的文本中提供。

5

CPU times: user 912 ms, sys: 152 ms, total: 1.06 s
Wall time: 4.37 s


In [20]:
%%time

streaming_response = query_engine.query("文中提到的局部真理是啥意思")
streaming_response.print_response_stream()
display(len(streaming_response.source_nodes))

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
文中提到的“局部的真理”可能是指一种不全面或不完整的理解或表达，它可能是在特定情境或视角下的真实情况，但并不是全面或绝对的真理。在文学作品中，这可能用来形容人物或事件的某个方面，或者是一种暂时的、相对的观点。由于没有具体的上下文，这里的“局部真理”可能是指鲍小姐并未完全裸露，而是一个形象的比喻或描述。

5

CPU times: user 963 ms, sys: 152 ms, total: 1.11 s
Wall time: 4.73 s
